# Set Up

In [1]:
!wget --passive-ftp --ftp-user FoodImage@grebvm2.epfl.ch --ftp-password Cahc1moo ftp://tremplin.epfl.ch/Food-11.zip

--2019-06-08 03:49:08--  ftp://tremplin.epfl.ch/Food-11.zip
           => ‘Food-11.zip’
Resolving tremplin.epfl.ch (tremplin.epfl.ch)... 128.178.50.75, 2001:620:618:132:1:80b2:324b:1
Connecting to tremplin.epfl.ch (tremplin.epfl.ch)|128.178.50.75|:21... connected.
Logging in as FoodImage@grebvm2.epfl.ch ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD not needed.
==> SIZE Food-11.zip ... 1163037552
==> PASV ... done.    ==> RETR Food-11.zip ... done.
Length: 1163037552 (1.1G) (unauthoritative)

Food-11.zip         100%[===================>]   1.08G   113MB/s    in 10s     

2019-06-08 03:49:19 (106 MB/s) - ‘Food-11.zip’ saved [1163037552]



In [2]:
!unzip Food-11.zip

Archive:  Food-11.zip
   creating: evaluation/
  inflating: evaluation/0_0.jpg      
  inflating: evaluation/0_1.jpg      
  inflating: evaluation/0_10.jpg     
  inflating: evaluation/0_100.jpg    
  inflating: evaluation/0_101.jpg    
  inflating: evaluation/0_102.jpg    
  inflating: evaluation/0_103.jpg    
  inflating: evaluation/0_104.jpg    
  inflating: evaluation/0_105.jpg    
  inflating: evaluation/0_106.jpg    
  inflating: evaluation/0_107.jpg    
  inflating: evaluation/0_108.jpg    
  inflating: evaluation/0_109.jpg    
  inflating: evaluation/0_11.jpg     
  inflating: evaluation/0_110.jpg    
  inflating: evaluation/0_111.jpg    
  inflating: evaluation/0_112.jpg    
  inflating: evaluation/0_113.jpg    
  inflating: evaluation/0_114.jpg    
  inflating: evaluation/0_115.jpg    
  inflating: evaluation/0_116.jpg    
  inflating: evaluation/0_117.jpg    
  inflating: evaluation/0_118.jpg    
  inflating: evaluation/0_119.jpg    
  inflating: evaluation/0_12.jpg     
  i

In [0]:
!mkdir dataset

In [0]:
import os
from imutils import paths
import shutil

In [0]:
train_path = 'training'
test_path = 'evaluation'
val_path = 'validation'

classes = [
    "Bread", "Dairy product", "Dessert",
    "Egg", "Fried food","Meat",
    "Noodles/Pasta", "Rice", "Seafood", 
    "Soup","Vegetable/Fruit"
]

batch_size = 32

# !mkdir output

model_path = os.path.sep.join(['output', 'food11.model'])

unfrozen_plot_path = os.path.sep.join(['output', 'unfrozen.png'])
warmup_plot_path = os.path.sep.join(['output', 'warmup.png'])

In [6]:
for s in (train_path, test_path, val_path):
  print('processing {}'.format(s))
  image_paths = list(paths.list_images(s))
  
  for im in image_paths:
    file_name = im.split(os.path.sep)[-1]
    label = classes[int(file_name.split("_")[0])]
    
    dir_path = os.path.sep.join(['dataset', s, label])
    if not os.path.exists(dir_path):
      os.makedirs(dir_path)
    new_loc = os.path.sep.join([dir_path, file_name])
    shutil.move(im, new_loc)

processing training
processing evaluation
processing validation


In [7]:
!ls dataset

evaluation  training  validation


In [8]:
!ls dataset/training

 Bread		  Dessert  'Fried food'   Noodles   Seafood   Vegetable
'Dairy product'   Egg	    Meat	  Rice	    Soup


# Training

## using vgg16

In [9]:
import matplotlib
matplotlib.use("Agg")

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers.core import Dropout, Flatten, Dense
from keras.layers import Input

from keras.models import Model
from keras.optimizers import SGD

from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

Using TensorFlow backend.


In [0]:
def plot_training(H, N, plot_path):
  """Function to plot the training graph."""
  plt.style.use("ggplot")
  plt.figure()
  
  for i in zip(['loss', 'val_loss', 'acc', 'val_acc'], ['train_loss', 'val_loss', 'train_acc', 'val_acc']):
    plt.plot(np.arange(0, N), H.history[i[0]], label=i[1])
  plt.title("Training Loss and accuracy")
  plt.xlabel("Epoch #")
  plt.ylabel("Loss / ACC")
  plt.legend(loc="lower left")
  plt.savefig(plot_path)

In [11]:
total_train = len(list(paths.list_images('dataset/training')))
total_val = len(list(paths.list_images('dataset/validation')))
total_test = len(list(paths.list_images('dataset/evaluation')))

total_train, total_val, total_test

(9866, 3430, 3347)

In [0]:
# data augmentation

train_aug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_aug = ImageDataGenerator()

mean = np.array([123.68, 116.779, 103.939], dtype="float32")

train_aug.mean = mean
val_aug.mean = mean

In [13]:
# data generator

train_gen = train_aug.flow_from_directory(
    'dataset/training',
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size
)

val_gen = val_aug.flow_from_directory(
    'dataset/validation',
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size
)

test_gen = val_aug.flow_from_directory(
    'dataset/evaluation',
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size
)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
Found 3347 images belonging to 11 classes.


In [14]:
base_model = VGG16(
    weights="imagenet",
    include_top=False,
    input_tensor=Input(shape=(224, 224, 3))
)

head_model = base_model.output
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(512, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(len(classes), activation="softmax")(head_model)

model = Model(
    inputs=[base_model.input],
    outputs=[head_model]
)

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 2s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
for layer in base_model.layers:
  layer.trainable = False

In [0]:
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(
    loss="categorical_crossentropy",
    optimizer=opt,
    metrics=["accuracy"]
)

In [17]:
H = model.fit_generator(
    train_gen,
    steps_per_epoch=total_train // batch_size,
    validation_data=val_gen,
    validation_steps=total_val // batch_size,
    epochs=50,
    verbose=2  
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 167s - loss: 10.2459 - acc: 0.3132 - val_loss: 7.3570 - val_acc: 0.4845
Epoch 2/50
 - 157s - loss: 8.0365 - acc: 0.4411 - val_loss: 5.7199 - val_acc: 0.5786
Epoch 3/50
 - 155s - loss: 6.0609 - acc: 0.5185 - val_loss: 3.2961 - val_acc: 0.6507
Epoch 4/50
 - 155s - loss: 2.5460 - acc: 0.4819 - val_loss: 1.4049 - val_acc: 0.5974
Epoch 5/50
 - 154s - loss: 1.7306 - acc: 0.4767 - val_loss: 1.3045 - val_acc: 0.6192
Epoch 6/50
 - 155s - loss: 1.6058 - acc: 0.4974 - val_loss: 1.2269 - val_acc: 0.6398
Epoch 7/50
 - 155s - loss: 1.5215 - acc: 0.5146 - val_loss: 1.1828 - val_acc: 0.6510
Epoch 8/50
 - 153s - loss: 1.4792 - acc: 0.5286 - val_loss: 1.1383 - val_acc: 0.6466
Epoch 9/50
 - 154s - loss: 1.4121 - acc: 0.5427 - val_loss: 1.1174 - val_acc: 0.6589
Epoch 10/50
 - 154s - loss: 1.3681 - acc: 0.5553 - val_loss: 1.0621 - val_acc: 0.6725
Epoch 11/50
 - 154s - loss: 1.3144 - acc: 0.5747 - val_loss: 1.0674 - val_acc: 0.6778
Epoch 12/50
 -

In [0]:
test_gen.reset()

pred_idxs = model.predict_generator(
    test_gen,
    steps=(total_test // batch_size) + 1
)

In [22]:
preds = [np.argmax(i) for i in pred_idxs]
preds[:5]

[0, 0, 3, 0, 0]

In [23]:
print(classification_report(test_gen.classes, preds, target_names=test_gen.class_indices.keys()))

               precision    recall  f1-score   support

        Bread       0.75      0.57      0.65       368
Dairy product       0.67      0.66      0.67       148
      Dessert       0.73      0.67      0.70       500
          Egg       0.64      0.75      0.69       335
   Fried food       0.70      0.72      0.71       287
         Meat       0.75      0.85      0.80       432
      Noodles       0.98      0.93      0.95       147
         Rice       0.90      0.90      0.90        96
      Seafood       0.82      0.80      0.81       303
         Soup       0.92      0.94      0.93       500
    Vegetable       0.86      0.90      0.88       231

     accuracy                           0.78      3347
    macro avg       0.79      0.79      0.79      3347
 weighted avg       0.78      0.78      0.78      3347



In [0]:
!mkdir output

In [0]:
plot_training(H, 50, warmup_plot_path)

In [0]:
import matplotlib.image as mpimg

In [0]:
img = mpimg.imread(warmup_plot_path)
plt.imshow(img)
plt.show()

In [27]:
train_gen.reset()
val_gen.reset()


for layer in base_model.layers[15:]:
  layer.trainable = True

for layer in base_model.layers:
  print("{}: {}".format(layer, layer.trainable))

<keras.engine.input_layer.InputLayer object at 0x7f1d02aa7e80>: False
<keras.layers.convolutional.Conv2D object at 0x7f1d02aa7b38>: False
<keras.layers.convolutional.Conv2D object at 0x7f1cb21f80f0>: False
<keras.layers.pooling.MaxPooling2D object at 0x7f1cb2219390>: False
<keras.layers.convolutional.Conv2D object at 0x7f1cb2219710>: False
<keras.layers.convolutional.Conv2D object at 0x7f1cb21c94e0>: False
<keras.layers.pooling.MaxPooling2D object at 0x7f1cb21e1ba8>: False
<keras.layers.convolutional.Conv2D object at 0x7f1cb21e1710>: False
<keras.layers.convolutional.Conv2D object at 0x7f1cb218c3c8>: False
<keras.layers.convolutional.Conv2D object at 0x7f1cb21a3e10>: False
<keras.layers.pooling.MaxPooling2D object at 0x7f1cb214f6a0>: False
<keras.layers.convolutional.Conv2D object at 0x7f1cb214f1d0>: False
<keras.layers.convolutional.Conv2D object at 0x7f1cb2101358>: False
<keras.layers.convolutional.Conv2D object at 0x7f1cb2117978>: False
<keras.layers.pooling.MaxPooling2D object at 0

In [34]:
# re compiling the model

print("Recompiling the model...")

opt = SGD(lr=1e-4, momentum=0.9)

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

H = model.fit_generator(
    train_gen,
    steps_per_epoch=total_train // batch_size,
    validation_data=val_gen,
    validation_steps=total_val // batch_size,
    epochs=20
)

Recompiling the model...
Epoch 1/20
308/308 [==============================] - 169s 547ms/step - loss: 0.8426 - acc: 0.7235 - val_loss: 0.7641 - val_acc: 0.7830
Epoch 2/20
308/308 [==============================] - 159s 517ms/step - loss: 0.7815 - acc: 0.7464 - val_loss: 0.7199 - val_acc: 0.7916
Epoch 3/20
308/308 [==============================] - 160s 519ms/step - loss: 0.7279 - acc: 0.7611 - val_loss: 0.7507 - val_acc: 0.7990
Epoch 4/20
308/308 [==============================] - 159s 515ms/step - loss: 0.6858 - acc: 0.7791 - val_loss: 0.7542 - val_acc: 0.7978
Epoch 5/20
308/308 [==============================] - 159s 516ms/step - loss: 0.6478 - acc: 0.7860 - val_loss: 0.6685 - val_acc: 0.8114
Epoch 6/20
308/308 [==============================] - 158s 513ms/step - loss: 0.5790 - acc: 0.8049 - val_loss: 0.6520 - val_acc: 0.8178
Epoch 7/20
308/308 [==============================] - 159s 515ms/step - loss: 0.5805 - acc: 0.8120 - val_loss: 0.6459 - val_acc: 0.8302
Epoch 8/20
308/308 [===

In [35]:
print("Evaluation after fine tuning network...")

test_gen.reset()

pred_idxs = model.predict_generator(
    test_gen,
    steps=(total_test // batch_size) + 1
)

pred_idxs = np.argmax(pred_idxs, axis=1)

print(classification_report(test_gen.classes, pred_idxs, target_names=test_gen.class_indices.keys()))

plot_training(H, 20, unfrozen_plot_path)

model.save(model_path)

Evaluation after fine tuning network...
               precision    recall  f1-score   support

        Bread       0.86      0.76      0.80       368
Dairy product       0.82      0.75      0.78       148
      Dessert       0.85      0.77      0.81       500
          Egg       0.79      0.87      0.83       335
   Fried food       0.80      0.88      0.84       287
         Meat       0.86      0.91      0.88       432
      Noodles       0.99      0.97      0.98       147
         Rice       0.93      0.94      0.93        96
      Seafood       0.86      0.92      0.89       303
         Soup       0.99      0.94      0.96       500
    Vegetable       0.92      0.97      0.94       231

     accuracy                           0.87      3347
    macro avg       0.88      0.88      0.88      3347
 weighted avg       0.87      0.87      0.87      3347

